In [ ]:
%run -i datapy/analyze/classes.py
%run -i datapy/analyze/descriptives.py
%run -i datapy/analyze/recomendation.py
%run -i datapy/analyze/server.py

In [ ]:
igor_path='C:\\Users\\Admin\\Desktop\\strategy_result.csv'
igor_path='C:\\temp\\strategy_full.csv'
anton_path='C:\\Users\\Anton\\repository\\datapy\\data\\strategy_full.csv'
path=anton_path

In [ ]:
%run -i datapy/analyze/server.py
s=server()
s.setfile(path,delimiter=';', decimal=',',encoding='1251')

## Определение ролей

In [ ]:
# начальное
types_and_roles, missing= s.PredictTypesAndRoles()
missing

In [ ]:
for i in range(len(types_and_roles)):
    print(i, types_and_roles[i].name, types_and_roles[i].type, types_and_roles[i].role)

In [ ]:
#Пользовательская корректировка

types_and_roles[0].role = VariableRoleEnum.ID
types_and_roles[1].role = VariableRoleEnum.DATE
for i in range(2, 21):
    types_and_roles[i].role = VariableRoleEnum.REJECTED
for i in range(22, 35):
    types_and_roles[i].role = VariableRoleEnum.FIXED_RULE
for i in range(35, 39):
    types_and_roles[i].role = VariableRoleEnum.HISTORICAL_RULE
for i in range(39, 41):
    types_and_roles[i].role = VariableRoleEnum.POTENCIAL_RULE


types_and_roles[21].role = VariableRoleEnum.TARGET
types_and_roles[20].role = VariableRoleEnum.NEW_CREDIT
types_and_roles[41].role = VariableRoleEnum.APPROVED
types_and_roles[42].role = VariableRoleEnum.NTU
types_and_roles[43].role = VariableRoleEnum.CREDITED

for i in range(44,len(types_and_roles)): 
    types_and_roles[i].role = VariableRoleEnum.REJECTED

for i in range(51,len(types_and_roles)): 
    types_and_roles[i].role = VariableRoleEnum.VALUE_COLUMN

types_and_roles_edited=types_and_roles
data_desctiption, data_desctiption_m, one_factor, unique_factor, time, ch_rules, p_rule = s.AnalyseMarkedData(types_and_roles_edited)
#вывод таблицы с расчетным временем
time

In [ ]:
#вывод таблицы со статистикой
data_desctiption

In [ ]:
#вывод таблицы со статистикой
data_desctiption_m

In [ ]:
#вывод таблицы с однофакторным анализом
one_factor

In [ ]:
#вывод таблицы с анализом уникальных отказов
unique_factor

# Пользователь выбирает параметры
после этого должен быть шаг выбора параметров:
опция 1: максимальный возможный уровень одобрения, при уровне риска не выше X% (уровень риска по одобренным или уровень риска по выданным - в зависимости от того, предоставлено ли поле NTU)
опция 2: минимальный уровень риска, при уровне одобрения не ниже Y%

In [ ]:
#поиск всех возможных комбинаций
found_combinations =s.FindCombinations()

In [ ]:
#поиск комбинаций по параметрам
# 1: максимальный возможный уровень одобрения, при уровне риска не выше X% (уровень риска по одобренным
param_max_dr=15
optimized_by_dr=s.OptimezeRulesDR(param_max_dr)
dr = pd.DataFrame(optimized_by_dr)

# 2: минимальный уровень риска, при уровне одобрения не ниже Y%
param_min_ar=10
optimized_by_ar=s.OptimezeRulesAR(param_min_ar)
ar = pd.DataFrame(optimized_by_ar)

In [ ]:
dr

# Пользователь вводит своё собственное правило

In [ ]:
users_combination=['r_limit_loanamount', 'r_limit_loanburden', 'r_verification', 'r_security']
user_combs=s.FindUsersCombination(users_combination)

In [ ]:
user_combs.head(1)

# Поиск однофакторных правил

In [ ]:
# метод для визуализации найденных правил
def RuleToText(rule):
    column_name=rule[0]
    thr=rule[1]
    trend=rule[2]
    dr_delta=rule[3]
    if trend is None:
        return "Для поля {} не было найдено подходящих правил".format(column_name)
    elif trend:
        return "Рекомендуется ограничить выдачи по тем заявкам, где {} выше {}. Что позволит снизить DR на {}%".format(column_name, thr, round(dr_delta*100,2))
    else:
        return "Рекомендуется ограничить выдачи по тем заявкам, где {} ниже {}. Что позволит снизить DR на {}%".format(column_name, thr, round(dr_delta*100,2))

In [ ]:
#Если пользователь не передал ничего - анализируем всё
full_onefactor_rules=s.RecomendOneFactorRules()

In [ ]:
for rule in full_onefactor_rules:
    print(RuleToText(rule))

In [ ]:
#Если пользователь передал набор полей - анализируем по ним
user_columns=['CalculatedMaxLoanAmount','CalculatedMaxLoanAmountCS','CalculatedMaxLoanAmountRUR','ClientJobTerm','ClientCreditPayment']
user_onefactor_rules=s.RecomendOneFactorRules(user_columns)

In [ ]:
for rule in user_onefactor_rules:
    print(RuleToText(rule))

In [ ]:
#приводим к бинарному виду
new_types_and_roles, potential_rules=s.Bin_new_rules(full_onefactor_rules)

In [ ]:
#повторно получаем статистику, теперь уже с новыми маркерами
new_data_desctiption, new_data_desctiption_m, new_one_factor, new_unique_factor, new_time, new_ch_rules, new_p_rule = s.AnalyseMarkedData(new_types_and_roles)

In [ ]:
new_one_factor

In [ ]:
#добавились новые правила
for i in range(len(types_and_roles)):
    print(i, types_and_roles[i].name, types_and_roles[i].type, types_and_roles[i].role)

In [ ]:
#поиск всех возможных комбинаций
found_combinations =s.FindCombinations()

In [ ]:
#поиск комбинаций по параметрам
# 1: максимальный возможный уровень одобрения, при уровне риска не выше X% (уровень риска по одобренным
param_max_dr=15
optimized_by_dr=s.OptimezeRulesDR(param_max_dr)
dr = pd.DataFrame(optimized_by_dr)

# 2: минимальный уровень риска, при уровне одобрения не ниже Y%
param_min_ar=10
optimized_by_ar=s.OptimezeRulesAR(param_min_ar)
ar = pd.DataFrame(optimized_by_ar)

In [ ]:
dr

# Поиск многофакторных правил

In [ ]:
# метод для визуализации найденных мульти-правил
def MultyRuleToText(rule, delta_dr):
    if rule is None:
        return 'Не было найдено подходящих правил. Проверьте, возможно один из столбцов пустой'
    result_text='Рекомендуется ограничить выдачи по заявкам, где '
    for r in rule[:-1]:
        result_text+='{} {} {} и '.format(r[0], r[1], round(r[2],4))
    result_text = result_text[:-3] + '. Что позволит снизить DR на {}%'.format(round(delta_dr*100, 2))
    return result_text

In [ ]:
#Если пользователь не передал ничего - анализируем всё
best_multirule, dr_delta=s.RecomendMultyFactorRules()
MultyRuleToText(best_multirule, dr_delta)

In [ ]:
#Если пользователь передал набор полей - анализируем по ним
user_columns=['CalculatedMaxLoanAmount','CalculatedMaxLoanAmountCS','CalculatedMaxLoanAmountRUR', 'ClientJobTermTotal']
user_best_multirule, user_dr_delta=s.RecomendMultyFactorRules(user_columns)
MultyRuleToText(user_best_multirule, user_dr_delta)

In [ ]:
ncr_CalculatedMaxLoanAmountRUR VariableTypeEnum.Binary VariableRoleEnum.POTENCIAL_RULE
97 ncr_ClientAge VariableTypeEnum.Binary VariableRoleEnum.POTENCIAL_RULE
98 ncr_ClientJobTerm VariableTypeEnum.Binary VariableRoleEnum.POTENCIAL_RULE
99 ncr_ClientJobTermTotal VariableTypeEnum.Binary VariableRoleEnum.POTENCIAL_RULE
100 ncr_ClientKidsNumber VariableTypeEnum.Binary VariableRoleEnum.POTENCIAL_RULE
101 ncr_FbrFicoScore VariableTypeEnum.Binary VariableRoleEnum.POTENCIAL_RULE
102 ncr_FbrNumAcc VariableTypeEnum.Binary VariableRoleEnum.POTENCIAL_RULE
103 ncr_FbrNumAccOpen VariableTypeEnum.Binary VariableRoleEnum.POTENCIAL_RULE
104 ncr_FbrNumAccOpen6M VariableTypeEnum.Binary VariableRoleEnum.POTENCIAL_RULE
105 ncr_FbrNumRequestCred3M VariableTypeEnum.Binary VariableRoleEnum.POTENCIAL_RULE
106 ncr_FbrSumCloseCredAll VariableTypeEnum.Binary VariableRoleEnum.POTENCIAL_RULE
107 ncr_FbrSumCredLimAll VariableTypeEnum.Binary VariableRoleEnum.POTENCIAL_RULE
108 ncr_FbrSumMonthPayAll VariableTypeEnum.Binary VariableRoleEnum.POTENCIAL_RULE
109 ncr_FICOScorePointValue VariableTypeEnum.Binary VariableRoleEnum.POTENCIAL_RULE
110 ncr_LoanBalance VariableTypeEnum.Binary VariableRoleEnum.POTENCIAL_RULE
111 ncr_LoanRate VariableTypeEnum.Binary VariableRoleEnum.POTENCIAL_RULE
112 ncr_LoanTerm VariableTypeEnum.Binary VariableRoleEnum.POTENCIAL_RULE
113 ncr_LoanYearRate VariableTypeEnum.Binary VariableRoleEnum.POTENCIAL_RULE
114 ncr_CalcClientAverageExpenses VariableTypeEnum.Binary VariableRoleEnum.POTENCIAL_RULE
115 ncr_CalcPreliminaryCostLoan VariableTypeEnum.Binary VariableRoleEnum.POTENCIAL_RULE
116 ncr_CalcMaxLoanCS VariableTypeEnum.Binary VariableRoleEnum.POTENCIAL_RULE
117 ncr_CalcMaxCostsCS VariableTypeEnum.Binary VariableRoleEnum.POTENCIAL_RULE
118 ncr_CalcMaxPaymCS VariableTypeEnum.Binary VariableRoleEnum.POTENCIAL_RULE
119 ncr_CalcCalculatedMaxLoanAmount VariableTypeEnum.Binary VariableRoleEnum.POTENCIAL_RULE
120 ncr_CalcScore VariableTypeEnum.Binary VariableRoleEnum.POTENCIAL_RULE